**Extract Flow Accumulation at Bogs**

Adrian Wiegman

2024-08-25

-------

## Description

7. Generate Bog Pour Points
    - find maximum flow accumulation value inside each bog.
    - generate a point at each bog maximum value.
    - identity to get cranberry bog attributes at each point.
8. Delineate Basins for each point
    - with the bog cranberry points loop through each point
        - delineate watershed using the bog pour point and the D8 flow direction
        - save the output to a temp file with the feature ID ('FID') of the cranberry bog. 
    - merge all cranberry bog basins to one polygon layer, containing the FID of the cranberry bog. 

## Setup Environment

In [1]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = r"C:\Users\Adrian.Wiegman\Documents\GitHub\Wiegman_USDA_ARS\Cran_Q_C\2_gis\scripts\_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook 

***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_run_script_w_propy_bat
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups
   fn_buildWhereClauseFromList
   fn_FA_to_Q
   fn_alter_field_double
   fn_return_float
   fn_classify_wetlands

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the source information about a given function name
    '''
    get_ipython().run_line_magic('pinf

In [2]:
# Create new file geodatabase to store results\n",
gdb = "Default.gdb"

ap.env.workspace = os.path.join(wdr,gdb)

## Flow Accumulation

In [3]:
# EXTRACT MAX FLOW ACCUMULATION WITHIN EACH BOG
names = [
"FA_D8_lidbr", #lid - burn_flowline_raw
"FA_D8_lidb", #lid - burn_flowline
"FA_D8_lidbc", #lid - burn_combined
"FA_D8_lidbcr", #lid - burn_combined
"FA_D8_gwebr", #gwe - burn_flowline_raw
"FA_D8_gweb", #gwe - burn_flowline
"FA_D8_gwebc", #gwe - burn_combined
"FA_D8_gwebcr", #gwe - burn_combined_raw
"FA_D8_gweblidbk1", #gweb*(1-0.01) + lidb*0.01
"FA_D8_gwebclidbck1", #gwebc*(1-0.01) + lidbc*0.01
"FA_D8_gweblidbk5", #gweb*(1-0.05) + lidb*0.05
"FA_D8_gwebrlidbrk1", #gwebr*(1-0.01) + lidbr*0.01
"FA_D8_gwebcrlidbcrk1", #gwebcr*(1-0.01) + lidbcr*0.01
"FA_D8_gwebrlidbrk5", #gwebr*(1-0.05) + lidbr*0.05
"FA_D8_gwebcrlidbcrk5", #gwebcr*(1-0.05) + lidbcr*0.05
"FA_D8_gweblidbk1_lid1", #(gweblidbk1 + 1)*gwd_ge1 + lidb*gwd_lt1 - 1 - burn_flowline
"FA_D8_gweblidbk1_lid0", #(gweblidbk1 + 0)*gwd_ge1 + lidb*gwd_lt0 - 0 - burn_flowline
"FA_D8_gweblidbk5_lid1", #(gweblidbk5 + 1)*gwd_ge1 + lidb*gwd_lt1 - 1 - burn_flowline
"FA_D8_gweblidbk5_lid0", #(gweblidbk5 + 0)*gwd_ge1 + lidb*gwd_lt0 - 0 - burn_flowline
"FA_D8_gwelidr1", #(gwebr + 1)*gwd_ge1 + lidbr*gwd_lt1 - 1 - burn_flowline
"FA_D8_gwelidr0", #(gwebr + 0)*gwd_ge0 + lidbr*gwd_lt0 - 0 - burn_flowline
"FA_D8_gwelid1", #(gweb + 1)*gwd_ge1 + lidb*gwd_lt1 - 1 - burn_flowline
"FA_D8_gwelid0", #(gweb + 0)*gwd_ge0 + lidb*gwd_lt0 - 0 - burn_flowline
]
files = [os.path.join(gdb_default_path,n) for n in names]
print(files)

['C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_lidbr', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_lidb', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_lidbc', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_lidbcr', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebr', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gweb', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebc', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebcr', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gweblidbk1', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebclidbck1', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gweblidbk5', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebrlidbrk1', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebcrlidbcrk1', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebrlidbrk5', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\FA_D8_gwebcrlidbcrk5', 'C:\\Workspace\\Geod

In [4]:
# make a copy of "bogs_agg_raw_SummarizeWithin": bogs that have been split, aggregated, and repopulated with original site info
selection = arcpy.management.SelectLayerByLocation(
    in_layer="bogs_agg_raw_SummarizeWithin",
    overlap_type="INTERSECT",
    select_features=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\domain_poly",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

arcpy.management.CopyFeatures(selection,"bogs_agg_FA")

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_agg_FA'>

In [5]:
in_feat = "bogs_agg_FA"
for i in range(len(files)):
    print(files[i])
    out_raster = arcpy.ia.ZonalStatistics(
        in_zone_data=in_feat,
        zone_field="OBJECTID",
        in_value_raster=files[i],
        statistics_type="MAXIMUM",
        ignore_nodata="DATA",
        process_as_multidimensional="CURRENT_SLICE",
        percentile_value=90,
        percentile_interpolation_type="AUTO_DETECT",
        circular_calculation="ARITHMETIC",
        circular_wrap_value=360)
    out_rast_name = "ZS_MAX_"+names[i]
    out_raster.save(out_rast_name)   

C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_lidbr
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_lidb
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_lidbc
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_lidbcr
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gwebr
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gweb
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gwebc
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gwebcr
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gweblidbk1
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gwebclidbck1
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gweblidbk5
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gwebrlidbrk1
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gwebcrlidbcrk1
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gwebrlidbrk5
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gwebcrlidbcrk5
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gweblidbk1_lid1
C:\Workspace\Geodata\Cran_Q_C\Default.gdb\FA_D8_gweblidbk1_lid0
C:\Workspace\

## Watershed

use the watershed tool to extract the "reach" watersheds, groundwater discharge that accumulates along the bog reach that did not previously flow through another bog. 

In [6]:
## Generate pour points for selected DEM
# SELECT DEM 
# use the DEM from the list above with the best fit for observed vs fitted after analyzing data 
dem = "gwelid0"

# Create raster of points were flow accumulation equals max values inside each bog.
a = arcpy.Raster("ZS_MAX_FA_D8_"+dem)
b = arcpy.Raster("FA_D8_"+dem)
c = a == b
c.save(r"c:\workspace\Geodata\Cran_Q_C\Default.gdb\zs_bog_pour_point_rast")

inras = 'zs_bog_pour_point_rast'
x = arcpy.sa.SetNull(inras, inras,'Value=0')
x.save(inras+"_NULL")

inras = 'zs_bog_pour_point_rast'
# convert to point
arcpy.conversion.RasterToPoint(
    in_raster=inras+"_NULL",
    out_point_features=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\pour_points_bogs",
    raster_field="Value"
)

# COMPUTE REACH WATERSHED BOUNDARIES FOR ALL BOGS
out_raster = arcpy.sa.Watershed(
    in_flow_direction_raster=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\D8_"+dem,
    in_pour_point_data="pour_points_bogs",
    pour_point_field="pointid"
    )
out_raster.save(r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\watershed_D8_"+dem+"_bogs")

arcpy.conversion.RasterToPolygon(
    in_raster="watershed_D8_"+dem+"_bogs",
    out_polygon_features=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\watersheds_D8_"+dem+"_bogs",
    simplify="NO_SIMPLIFY",
    raster_field="OBJECTID",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

arcpy.management.Dissolve(
    in_features=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\watersheds_D8_"+dem+"_bogs",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\watersheds_D8_"+dem+"_bogs_Dissolve",
    dissolve_field="gridcode",
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

arcpy.management.CalculateGeometryAttributes(
    in_features="watersheds_D8_"+dem+"_bogs_Dissolve",
    geometry_property="marg_bogshed_area_ha AREA",
    area_unit="HECTARES",
    coordinate_system=None,
    coordinate_format="SAME_AS_INPUT"
)

# make a copy of bogs
arcpy.management.CopyFeatures("pour_points_bogs","bogs_pour_points")
arcpy.management.JoinField(
    in_data="bogs_pour_points",
    in_field="pointid",
    join_table=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\watersheds_D8_"+dem+"_bogs_Dissolve",
    join_field="gridcode",
    fields="marg_bogshed_area_ha ",
    fm_option="NOT_USE_FM",
    field_mapping=None,
    index_join_fields="NO_INDEXES"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points'>

In [7]:
arcpy.management.CalculateField(
    in_table=r"bogs_pour_points",
    field="marg_bogshed_area_m2",
    expression="!marg_bogshed_area_ha!*10000",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points'>

In [8]:
# Add X and Y coordinates to table
arcpy.management.CalculateGeometryAttributes(
    in_features=r"bogs_pour_points",
    geometry_property="Lat POINT_X;Long POINT_Y",
    length_unit="",
    area_unit="",
    coordinate_system='PROJCS["NAD_1983_UTM_Zone_19N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-69.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    coordinate_format="DD"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points'>

In [9]:
# generate string of rasters to extract
long_names = ["ZS_MAX_"+n for n in names]
long_names
_ = ["{} {}".format(os.path.join(wdr,gdb,n),n) for n in long_names]
in_rasters = ";".join(_)
print(in_rasters)

C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_lidbr ZS_MAX_FA_D8_lidbr;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_lidb ZS_MAX_FA_D8_lidb;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_lidbc ZS_MAX_FA_D8_lidbc;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_lidbcr ZS_MAX_FA_D8_lidbcr;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gwebr ZS_MAX_FA_D8_gwebr;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gweb ZS_MAX_FA_D8_gweb;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gwebc ZS_MAX_FA_D8_gwebc;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gwebcr ZS_MAX_FA_D8_gwebcr;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gweblidbk1 ZS_MAX_FA_D8_gweblidbk1;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gwebclidbck1 ZS_MAX_FA_D8_gwebclidbck1;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gweblidbk5 ZS_MAX_FA_D8_gweblidbk5;C:\Workspace\Geodata\Cran_Q_C\Default.gdb\ZS_MAX_FA_D8_gwebrlidbrk1 ZS_MAX_FA_D8_gwebr

In [10]:
arcpy.sa.ExtractMultiValuesToPoints(
    in_point_features="bogs_pour_points",
    in_rasters=in_rasters,
    bilinear_interpolate_values="NONE")

<geoprocessing server result object object at 0x000002273DD3F870>

FA = Flow accumulation (number of cells draining to a point)
w = width of cell in distance units (10 meters)
l = length of cell in distance units (10 meters)

A = FA * cell_size^2

cell size is the side length of the grid cells

A = FA * 100 m^2/cell

Q = A * r

where r is recharge rate 
27.25 in/yr 

m3/d =  27.25 in/yr * 2.54 cm/in * 1/100 m/cm *  1/365.25 yr/d



In [11]:
??fn_FA_to_Q

Signature: fn_FA_to_Q(rasterpath=None, recharge_rate_in_yr=27.25)
Docstring: <no docstring>
Source:   
def fn_FA_to_Q (rasterpath=None,recharge_rate_in_yr = 27.25):
    _ = arcpy.GetRasterProperties_management(rasterpath, "CELLSIZEX")
    #Get the elevation standard deviation value from geoprocessing result object
    cellsize_x = _.getOutput(0)
    _ = arcpy.GetRasterProperties_management(rasterpath, "CELLSIZEY")
    cellsize_y = _.getOutput(0)
    # calculate cell area in meters
    cell_area_meters = float(cellsize_x) * float(cellsize_y)
    print("cell area {} square meters".format(cell_area_meters))
    FA_to_Q = cell_area_meters * recharge_rate_in_yr * 2.54 * (1/100) * (1/365.25)
    print("FA_to_Q = {} m3/d per cell".format(FA_to_Q))
    return(FA_to_Q)
File:      c:\windows\system32\<ipython-input-1-ca1a1ecab51d>
Type:      function


In [12]:
fn_FA_to_Q(rasterpath=os.path.join(wdr,gdb,"ZS_MAX_FA_D8_gwe_bf"))

cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell


0.1895003422313484

In [13]:
#%%capture # supress outputs 
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    # calculate flow for all flow accumulation layers
    # r"C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\bogs_points"
    in_table = os.path.join(wdr,gdb,"bogs_pour_points")
    for long_name in long_names:
        print(long_name)
        FA_to_Q = fn_FA_to_Q(rasterpath=os.path.join(wdr,gdb,long_name))
        _ = re.search("ZS_MAX_(.*)",long_name)
        short_name = re.sub("FA","",re.sub("_","",_[1]))
        print(short_name)
        arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=in_table,
            selection_type="NEW_SELECTION",
            where_clause="{} IS NOT NULL".format(long_name))
        arcpy.management.CalculateField(
            in_table=in_table,
            field="Q_m3d_{}".format(short_name),
            expression="!{}!*{}".format(long_name,FA_to_Q),
            expression_type="PYTHON3",
            code_block="",
            field_type="DOUBLE",
            enforce_domains="NO_ENFORCE_DOMAINS")
        arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=in_table,
            selection_type="CLEAR_SELECTION")

ZS_MAX_FA_D8_lidbr
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8lidbr
ZS_MAX_FA_D8_lidb
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8lidb
ZS_MAX_FA_D8_lidbc
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8lidbc
ZS_MAX_FA_D8_lidbcr
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8lidbcr
ZS_MAX_FA_D8_gwebr
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8gwebr
ZS_MAX_FA_D8_gweb
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8gweb
ZS_MAX_FA_D8_gwebc
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8gwebc
ZS_MAX_FA_D8_gwebcr
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8gwebcr
ZS_MAX_FA_D8_gweblidbk1
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d per cell
D8gweblidbk1
ZS_MAX_FA_D8_gwebclidbck1
cell area 100.0 square meters
FA_to_Q = 0.1895003422313484 m3/d 

In [14]:
# SELECT INTERPOLATED N CONCENTRATIONS

In [15]:
names = ["EBK_NO3_p25",
         "EBK_NO3_p50",
         "EBK_NO3_p75",
         "EBK_TN_p25",
         "EBK_TN_p50",
         "EBK_TN_p75",]
files = [os.path.join(gdb_default_path,n) for n in names]
print(files)

['C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_NO3_p25', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_NO3_p50', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_NO3_p75', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_TN_p25', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_TN_p50', 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\EBK_TN_p75']


In [16]:
arcpy.sa.ExtractMultiValuesToPoints(
    in_point_features="bogs_pour_points",
    in_rasters=files,
    bilinear_interpolate_values="NONE")

<geoprocessing server result object object at 0x000002273DD44C90>

In [17]:
arcpy.analysis.Identity(
    in_features="bogs_pour_points",
    identity_features="bogs_agg_FA",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg_raw",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points_ident_agg_raw'>

In [18]:
arcpy.analysis.Identity(
    in_features="bogs_pour_points_ident_agg_raw",
    identity_features="MEP_Embayments_Domain",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg_raw_MEP",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

arcpy.analysis.Identity(
    in_features="bogs_pour_points_ident_agg_raw_MEP",
    identity_features="Basins_ZS",
    out_feature_class=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS",
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS'>

In [19]:
# DELETE UNWANTED FIELDS
arcpy.management.DeleteField(
    in_table=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS",
    drop_field="FID_bogs_pour_points_ident_agg_raw_MEP;FID_bogs_pour_points_ident_agg_raw;FID_bogs_pour_points_ident_agg;FID_bogs_pour_points;pointid;grid_code;FID_bogs_agg_FA;ID;ORIG_FID_1;OBJECTID_1;OBJECTID_12;OBJECTID_12_13;OBJECTID_12_13_14;OBJECTID_12_13_14_15;OBJECTID_12_13_14_15_16;OBJECTID_12_13_14_15_16_17;OBJECTID_12_13_14_15_16_17_18;OBJECTID_12_13_14_15_16_17_18_19;OBJECTID_12_13_14_15_16_17_18_19_20;OBJECTID_12_13_14_15_16_17_18_19_20_21",
    method="DELETE_FIELDS")

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS'>

In [20]:
arcpy.management.DeleteField(
    in_table=r"C:\Workspace\Geodata\Cran_Q_C\Default.gdb\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS",
    drop_field="BUFF_DIST;imp_PCT25;imp_PCT75;pop_PCT25;pop_PCT75;NO3_p25_SUM",
    method="DELETE_FIELDS"
)

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\Default.gdb\\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS'>

In [21]:
points = "bogs_pour_points_ident_agg_raw_MEP_Basins_ZS"
arcpy.conversion.TableToExcel(
    Input_Table=points,
    Output_Excel_File=os.path.join(odr,points+"_extract_Q_C.xls"),
    Use_field_alias_as_column_header="NAME",
    Use_domain_and_subtype_description="CODE")

<Result 'C:\\Workspace\\Geodata\\Cran_Q_C\\outputs\\bogs_pour_points_ident_agg_raw_MEP_Basins_ZS_extract_Q_C.xls'>

## Appendix Unused Code